In [1]:
import numpy as np
from ml_models import OptimizationModel
def run_optimization():
    # Define random bounds for the variables
    bounds = [
        {'name': 'var1', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var2', 'type': 'continuous', 'domain': (0, 50)},
        {'name': 'var3', 'type': 'continuous', 'domain': (0, 50)}
    ]

    # Define a random target value
    target_value = np.random.randint(1, 200)
    print(target_value)

    # Define random reagent information
    reagent_info = {'reagent1': 20, 'reagent2': 30, 'reagent3': 25, 'reagent4': 50, 'reagent5': 15}  # Assuming volumes in μL

    # Define fixed reagents (randomly chosen)
    fixed_reagents = [('reagent2', 30), ('reagent3', 25), ('reagent4', 50)]  # Example fixed reagents with volumes

    # Initialize the optimization model
    opt_model = OptimizationModel(bounds, target_value, reagent_info, fixed_reagents, initial_design_numdata=15, batch_size=3, max_iters=40)

    def simulate_chemistry(x):
        # Define coefficients to simulate more nuanced chemistry behavior
        a, b, c = 1.0, 0.1, 0.05
        
        # Simulate an outcome with more predictable behavior
        result = a * x[:, 0] + b * x[:, 1]**2 - c * x[:, 2]
        
        # Add a small, controlled amount of noise to mimic experimental variation
        #noise = np.random.normal(0, 1, size=x.shape[0])
        return (result)
    def calc_obj(x):
        return abs(target_value - x).reshape(-1, 1)

    # Generate initial design and simulate experiments to get initial data
    X_initial = opt_model.generate_initial_design()
    Y_initial = calc_obj(simulate_chemistry(X_initial))

    print(type(X_initial))
    print(type(Y_initial))

    #print("x init", X_initial, 'yinit',Y_initial.shape)
    opt_model.experiment_data['X'] = list(X_initial)
    opt_model.experiment_data['Y'] = list(Y_initial)
    # Initialize the optimizer with initial experimental data
    opt_model.initialize_optimizer(X_initial, Y_initial)

    print(opt_model.optimizer)
    print(opt_model.acquisition)

    # Learning loop
    number_of_iterations = 75  # Define the number of iterations for the learning loop
    for i in range(number_of_iterations):
        # Suggest next locations for experiments
        next_locations = opt_model.suggest_next_locations()
        print(next_locations)
        
        # Simulate experiments at the suggested locations
        Y_new = calc_obj(simulate_chemistry(next_locations))
        
        # Update the optimizer with the new experimental data
        opt_model.update_experiment_data(next_locations, Y_new)
        
        print(f"Iteration {i+1}, Suggested Locations: {next_locations}, Results: {Y_new}")
        print(opt_model.quit, opt_model.curr_iter)
        
        # Optional: Check if the optimizer has met the target or other stopping criteria
        if opt_model.quit:
            print("Stopping criteria met.")
            break
    # To get the best observed X values (parameters)
    X_best = opt_model.optimizer.X[np.argmin(opt_model.optimizer.Y)]

    # To get the best observed Y value (function value)
    Y_best = np.min(opt_model.optimizer.Y)
    return i, target_value, (X_best, Y_best)


In [2]:
test_info = {}
for test in range(10):
    num_iter, tar, best = run_optimization()
    test_info[test] = (num_iter, tar, best)


98
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1
MPI
[[26.56208868 29.56181678 37.77535242]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 1, Suggested Locations: [[26.56208868 29.56181678 37.77535242]], Results: [[14.06342218]]
False 1
2
LCB
[[19.41553082 30.7305768  29.58372285]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 2, Suggested Locations: [[19.41553082 30.7305768  29.58372285]], Results: [[14.37317975]]
False 2
0
EI
[[35.57810145 17.4388662   0.        ]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 3, Suggested Locations: [[35.57810145 17.4388662   0.        ]], Results: [[32.01049312]]
False 3
1
MPI
[[50.        17.1738664 11.0368927]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 4, Suggested Locations: [[50.        17.1738664 11.0368927]], Results: [[19.05767593]]
False 4
2
LCB
[[50.  0.  0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Iteration 5, Suggested Locations: [[50.  0.  0.]], Results: [[48.]]
False 5
0


In [3]:
vals = test_info.values()
vals

dict_values([(5, 98, (array([17.97046688, 28.64446927, 38.09432372]), 0.11631268249320215)), (7, 93, (array([50.      , 20.624127,  0.      ]), 0.46453853790248445)), (7, 25, (array([26.63294458,  3.06711456, 39.82031259]), 0.5826481209260699)), (39, 156, (array([ 4.57603811, 39.23107235,  4.93736374]), 2.236873694785004)), (39, 185, (array([50.        , 37.33294629, 40.27647473]), 2.3610641570548694)), (15, 130, (array([27.86914206, 32.18941792, 44.98016033]), 0.7640033334231191)), (22, 181, (array([32.13800868, 38.66727225, 31.52652553]), 0.9225232615193022)), (20, 118, (array([18.29508961, 31.84411427, 24.43215913]), 0.4782430202650545)), (39, 80, (array([15.61458594, 25.90406045, 25.38394178]), 1.4474236367162092)), (13, 199, (array([50.        , 38.64710507,  6.53410133]), 0.03316794739851048))])

In [4]:
# These are the total number of iters before reaching some stopping criteria for each test
h = [val[0] for val in vals]
# These are the target values used in each test
i = [val[1] for val in vals]
print(h)
print(i)

# Print average nuber of iterations to meet stopping criteria with 3 variable parameters and max iters of 40:
print(sum(h)/len(h))

[5, 7, 7, 39, 39, 15, 22, 20, 39, 13]
[98, 93, 25, 156, 185, 130, 181, 118, 80, 199]
20.6
